# Twitter Streaming API

This notebook contains the methods required to perform a capture of tweets. It uses a list of words as a filter and captures only the tweets that contain at least one of the words in the list.

This list is the stream_args variable, and it can be modified to look for whatever is needed. In our case we have looked for a list of food terms that will give as a result a dataset of tweets for nutrition analysis in a certain area.

In [ ]:
#Importamos 
#Stream listener
from tweepy.streaming import StreamListener
#Autenticación de desarrollador de twitter
from tweepy import OAuthHandler
#Funciones de tratado de streams
from tweepy import Stream
#Api de twitter
from tweepy import API
#Funciones de tratado de tiempos
from time import gmtime, strftime, localtime
import logging, datetime
import time
#Funciones del sistema operativo
import os

#Clase FileListener, control
class FileListener(StreamListener):
    # Constructor
    def __init__(self, path, restart_time):
        self.path = path
        self.current_file = None
        self.restart_time = restart_time
        self.file_start_time = time.time()
        self.file_start_date = datetime.datetime.now()
        self.firstTweet = False

    # Me llegan los datos de un tweet
    def on_data(self, data):
        # Guardo la fecha
        current_time = datetime.datetime.now()
        # Creo un fichero nuevo si:
        # - No existe fichero anterior( inicializar)
        # - Si se ha superado el tiempo para restart
        # - Si se ha cambiado de dia
        if self.current_file == None or time.time() - self.restart_time > self.file_start_time \
                or self.file_start_date.day != current_time.day:
            self.startFile()
            self.file_start_date = datetime.datetime.now()
        # Escribo los datos en el fichero
        if self.firstTweet:
            self.current_file.write('[')
        if data.startswith('{'):
            if not self.firstTweet:
                self.current_file.write(',')
            self.current_file.write(data)
            self.firstTweet=False
            if not data.endswith('\n'):
                self.current_file.write('\n')

    def on_error(self, status_code):
        # Error 420: Rate Limited -> We want disconnect
        logger.error(status_code)
        if status_code == 420:
            exit()
            return False

    #Función que crea un nuevo archivo
    def startFile(self):
        #Si hay un archivo en marcha lo cerramos
        if self.current_file:
            self.current_file.write("]")
            self.current_file.close()
        #Cogemos la fecha actual y le damos formato
        local_time_obj = localtime()
        datetime = strftime("%Y_%m_%d_%H_%M_%S", local_time_obj)
        year = strftime("%Y", local_time_obj)
        month = strftime("%m", local_time_obj)
        day = strftime("%d", local_time_obj)
        #Creamos el sistema de directorios
        full_path = os.path.join(self.path, year)
        full_path = os.path.join(full_path, month)
        full_path = os.path.join(full_path, day)
        try:
            # Creo un directorio con la fecha de hoy
            os.makedirs(full_path)
            logger.info('Created %s' % full_path)
        except:
            pass
        # Creo el fichero de nombre: fecha y hora
        filename = os.path.join(full_path, '%s.json' % datetime)
        #self.current_file = gzip.open(filename, 'w')
        self.current_file = open(filename,'w')
        self.file_start_time = time.time()
        self.firstTweet=True
        logger.info('Starting new file: %s' % filename)



# Al lanzarlo como script
if __name__ == '__main__':
 
    consumer_key = 'euPYjsF2uwRGY12RVKmmFuWsf'
    consumer_secret = '64D7imPLpRF01qDcNics3Z76ZsJn2W8ZIfK7ToHFK6GPQLjxFV'
    access_token = '829766913647603714-Uy4x4YuK9czxrECfPmoQVwkMv5f0B2u'
    access_token_secret = 'XL6QLjtiUAKZsPaEvIqpsKwA3Rq92YFdqTSkBop7X0Egq'
    output_directory = 'tweetsEs'
    if not os.path.isdir('log'):
        os.makedirs('log')
    log_filename = 'log/logEs'

    logger = logging.getLogger('tweepy_streaming')
    handler = logging.FileHandler(log_filename, mode='a')
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


    #stream_args=['pizza',' té',' cafe',' helado',' crema agria',' pollo',' queso',' coca-cola',' huevo',' jamón',' tarta',' vino',' cerveza',' maiz',' arroz',' bacon',' sopa',' leche',' costillas',' guisante',' snack',' chuches',' salsa',' cereales',' plátano',' banana',' cookie',' galletas',' carne',' pescado',' fruta',' chocolate',' fast food',' comida rápida',' patatas',' cacahuetes',' avena',' ternera',' chile',' vainilla',' fresa',' mantequilla',' calabaza',' perrito caliente',' salchicha',' jengibre',' cerdo',' cubierta de chocolate',' patata',' pasta',' fresas',' crackers',' aceitunas',' patatas dulces',' ajo',' pavo',' yogur',' zumo',' pepinillos',' canela',' pimienta',' vegetales',' salmón',' piña',' noodles con pollo',' cangrejo',' limonada',' espinacas',' pepperoni',' mermelada',' mozzarela',' hamburguesa',' ostras',' zumo de naranja',' tomates',' coco',' sandwich de helado',' zanahorias',' leche de almendras',' trigo',' cheddar',' sandía',' cono de helado',' pimientos',' vegetal',' parmesano',' manzanas',' melocotones',' acelga',' lechuga',' menta',' sopa de tomate',' col rizada',' champiñones',' ensalada de patata',' pechuga de pollo',' ginger ale',' judías verdes',' uvas',' ensalada de fruta',' teriyaki',' cebolla',' raices',' pepino',' pretzels',' avena',' falda',' zanahorias',' calabacín',' judías verdes',' ancho',' salsa',' fresa y plátano',' ramen',' naranjas',' ganso',' brocoli',' bacon de pavo',' sopa de patata',' berenjena',' guisantes',' granola',' miel y mostaza',' anillos de cebolla',' peras',' mostaza',' sesamo',' paté',' ravioli',' lima',' frijoles',' mantequilla de almendra',' hombro',' ternera',' almendras',' avellana',' jalapeño',' cabra',' sprite',' leche de coco',' uvas',' nuez',' vinagre',' cerveza de raíz',' aguacate',' wasabi',' romero',' almendra',' nata',' aderezo para ensaladas',' ciruela',' olivas',' barra de helado',' yema de huevo',' sidra',' soda con helado',' red bull',' zimo de limón',' sirope de chocolate',' whiskey',' chorizo',' coliflor',' manteca de cacao',' coba',' vino tinto',' girasol',' barbacoa',' repollo',' oveja',' agua de coco',' arándanos',' arroz con pollo',' pastrami',' albahaca',' ricotta',' frambuesas',' zumo de manzana',' feta',' romano',' limones',' salsa de queso',' sopa de vegetales',' apio',' aguacates',' bebida isotónica',' mangos',' edamame',' salami',' fideos con pollo',' provolone',' pasas',' mayonesa',' especias',' aderezo ranchero',' edam',' crema de coco',' espárragos',' papaya',' coles de Bruselas',' ponche',' salsa de spaguetti',' pimiento morrón',' cerezas',' bacon canadiense',' monterey',' nueces',' arándanos',' filete de lomo',' boloñesa',' moras',' lomo',' lomo de cerdo',' sopa de almejas',' piel de patata',' potitos',' grasa de tocino',' plátanos',' bálsamo',' gouda',' hígado de pollo',' perejil',' cheese food',' crema de trigo',' caldo de pollo',' suero',' serrano',' cereales',' nestle',' aderezo italiano',' codorniz',' limonada',' azafrán',' salchichas de pavo',' zumo de piña',' tomillo',' sardina',' chukrut',' refresco de uva',' cola de buey',' hamburguesa de pollo',' zumo de lima',' cebollino',' kellogs',' hinojo',' tortitas de patata',' limas',' nueces pecanas',' riñones',' chalotes',' batido',' gruyere',' oregano',' gazpacho',' moras',' costilla',' sopa de guisantes',' lentejas',' higos',' sopa de queso y brocoli',' remolachas',' ruibarbo',' paprika',' estragón',' carne de cerdo',' sopa de tomate',' piñones',' carissa',' portabella',' rollo de pollo',' carne strogonoff',' extracto de vainilla',' semillas de mostaza',' sopa de cebolla',' salsa de arándanos',' berro',' chile en polvo',' dulce de leche',' ciruelas',' arenque',' rúcula',' queso de untar',' kielbasa',' cáscara de naranja',' hojas de mostaza',' pollo de corral',' daiquiri',' rábanos',' pavo molido',' zumo de uva',' pechuga de pavo',' semillas de sésamo',' piña colada',' chile picante',' savor vainilla',' copos de maiz',' sopa minestrone',' brocoli con queso',' semilla de amapola',' zumo de zanahoria',' nuez moscada',' gotas de miel',' aceite vegetal',' saboya',' queso de cerdo',' salchichas de cerdo',' noodles con carne',' pollo con champiñones',' crema de champiñones',' salchichas bratwurst',' crema de sopa de champiñones',' coles',' zumo de uva',' cúrcuma',' judías de lima',' molleja',' jamon de pavo',' granadas',' mandarinas',' sopa de fideos con carne',' hojas de nabo',' cocktail de frutas',' shiitake',' semillas de chia',' pollo y cerdo',' fabada',' sandwich de helado con cookies',' empanado',' nabos',' melones',' nectarinas',' quelpo',' carne de cerdo',' salsa worcestershire',' cheshire',' puerros',' crema de pollo',' alcaparras',' clavo',' chocolate en polvo',' raiz de jengibre',' faisán',' caquis',' melocotón y mango',' wakame',' pastinacas',' margarina',' radicchio',' harina',' calabaza',' flanco',' berro',' durian',' mandarinas',' hojas de uva',' zumo de tomate',' cuerda',' sopa de pollo con verduras',' sopa de tomate y nata',' alcachofas',' caldo de carne',' sopa de brocoli y crema',' menta',' pastel de calabaza',' cardamomo',' albóndigas de pollo',' sopa de queso y crema',' pollo gumbo',' sofrito',' ciruelas',' nueces de macadamia',' zumo de granada',' salchicha de ternera',' salchicha polaca',' mantequilla de anacardo',' sopa de almejas New England',' macarrones con queso',' albaricoques',' endibias',' pudín de maíz',' corazones de palma',' nopales',' mortadela',' soja',' solomillo',' aderezo ruso',' polvo de cebolla',' espirulina',' castañas',' refresco de naranja',' soja',' escarola',' sopa de carne con vegetales',' aderezo mil islas',' roquefort',' mil islas',' carne roja',' tequila sunrise',' whiskey amargo',' carne con verduras']
    stream_args=['#desayuno','#aperitivo','#comida','#brunch','#snack','#merienda','#cena']
    if not os.path.isdir('pid'):
        os.makedirs('pid')
    pid_file = 'pid/pidSpanish'
    file = open(pid_file, 'w')
    file.write(str(os.getpid()))
    file.close()

    #Creo el Listener:
    # - Con el directorio donde colocar los tweets
    # - Y con el tiempo de crear un fichero nuevo
    listener = FileListener(output_directory, 3600)
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    logger.warning("Connecting Process Spanish")
    api = API(auth)
    logger.warning(api.me().name)

    stream = Stream(auth=auth, listener=listener)
    stream.filter(track=stream_args)